In [ ]:
import anthropic
import sys
from defusedxml import ElementTree
from collections import defaultdict
import os
from typing import Any
import json

claude3_api_key = os.getenv("CLAUDE3_API_KEY")
client = anthropic.Client(api_key=claude3_api_key)

In [ ]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=claude3_api_key,
)
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content)

In [ ]:
# https://docs.anthropic.com/claude/docs/functions-external-tools
# https://medium.com/@zeek.granston/function-calling-with-anthropic-claude-and-amazon-bedrock-c6eda7358b0f
# https://github.com/anthropics/anthropic-tools
# https://levelup.gitconnected.com/anthropic-claude-v3-function-calling-2c7b5dc8f715

import requests

def get_weather(latitude: str, longitude: str):
  url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
  response = requests.get(url)
  return response.json()

get_weather_description = """
<tool_description>
<tool_name>get_weather</tool_name>
<description>
  Returns weather data for a given latitude and longitude. 
</description>
<parameters>
  <parameter>
  <name>latitude</name>
<type>string</type>
<description>The latitude coordinate as a string</description>
</parameter> <parameter>
<name>longitude</name>
<type>string</type>
<description>The longitude coordinate as a string</description>
</parameter>
</parameters>
</tool_description>
"""

def get_lat_long(place):
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': place, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params).json()
    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None

get_lat_long_description = """<tool_description>
<tool_name>get_lat_long</tool_name>
<description>
Returns the latitude and longitude for a given place name.
</description>
<parameters>
<parameter>
<name>place</name>  
<type>string</type>
<description>
The place name to geocode and get coordinates for.
</description>
</parameter>
</parameters>
</tool_description>"""

list_of_tools_specs = [get_weather_description, get_lat_long_description]

In [ ]:
lat_long = get_lat_long("San Jose")
print("latitude, longitude:  ",lat_long)

weather_data = get_weather(lat_long['latitude'], lat_long['longitude'])
print(weather_data)

In [ ]:
tools_string = "\n".join(list_of_tools_specs)

system_prompt = f"""
You have access to an additional set of one or more functions you can use to 
answer the user's question.

You may call them like this:
<function_calls>
    <invoke>
        <tool_name>$TOOL_NAME</tool_name>
        <parameters>
            <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
            ...
        </parameters>
    </invoke>
</function_calls>

Here are the tools available:
<tools>{tools_string}</tools>
Your final answer should be of the form 'The current temperature in <location> is <temp> degrees'
"""

print(system_prompt)

In [ ]:
client = anthropic.Anthropic(
    api_key=claude3_api_key
)

# set up our client message
#i.e what we want to ask the model
#
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    temperature=0.0,
    system=system_prompt,
    messages=[
        {"role": "user", "content": "Get lattitude and longitude for San Jose"}
    ]
)

print(message.content[0].text)